<a href="https://colab.research.google.com/github/limshaocong/ReqBERT/blob/main/Notebooks/t2_finetuning_seqclass_sc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminaries

The following libraries are required for this notebook. If run locally, it can be commented out.


* `transformers` - an excellent library by Hugging Face that provides a high-level abstraction for fine-tuning of transformer-based models, and a model hub of pre-trained models
* `datasets` - a dataset library by Hugging Face. All datasets used for this thesis are uploaded to this dataset hub. 
* `torch` - PyTorch. When running the `allenai/scibert_scivocab_cased` pre-trained model with this notebook on GCP Vertex AI, `torch` needs to be installed.

In [ ]:
! pip install --user datasets transformers torch

While Google Colab provides free GPU access, the specific GPU (K80/ T4/ P100) available is a function of the subscription level (Free Tier/ Colab Pro/ Colab Pro+) and usage trends. Alternatively, Google Cloud Platform provides US$300 in free credits that can be used for paid GPU access.

In [ ]:
! nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-038240b0-342b-0f63-d0c6-32ae64ef1e92)


# Import & Pre-process Data

These dictionaries will minimize the amount of adjustment within the remainder of the script.

In [ ]:
# This dict allows the correct tokenizer to be imported
# ReqBERT, ReqRoBERTa and ReqSciBERT uses the tokenizers from the
# corresponding base models
model_type_dict = {
    'bert-base-cased' : 'bert-base-cased',
    'roberta-base' : 'roberta-base',
    'allenai/scibert_scivocab_cased' : 'allenai/scibert_scivocab_cased',
    'limsc/reqbert-tapt-epoch29' : 'bert-base-cased',
    'limsc/reqroberta-tapt-epoch43' : 'roberta-base',
    'limsc/reqscibert-tapt-epoch20' : 'allenai/scibert_scivocab_cased',
}

model_name_dict = {
    'bert-base-cased' : 'bert',
    'roberta-base' : 'roberta',
    'allenai/scibert_scivocab_cased' : 'scibert',
    'limsc/reqbert-tapt-epoch29' : 'reqbert-e29',
    'limsc/reqroberta-tapt-epoch43' : 'reqroberta-e43',
    'limsc/reqscibert-tapt-epoch20' : 'reqscibert-e20'
}

task_name_dict = {
    'limsc/fr-nfr-classification' : 'frnfr',
    'limsc/subclass-classification' : 'subclass',
    'limsc/requirements-entity-recognition' : 'er'
}

The dataset for the [Task 2 - Subclass Classification](https://huggingface.co/datasets/limsc/subclass-classification) is uploaded onto the HuggingFace hub.

In [ ]:
from datasets import load_dataset

ds_name = 'limsc/subclass-classification'
ds = load_dataset(ds_name)
ds

Using custom data configuration limsc--subclass-classification-00abdf904b769dcf
Reusing dataset parquet (/root/.cache/huggingface/datasets/limsc___parquet/limsc--subclass-classification-00abdf904b769dcf/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['source', 'reqs', 'class', '__index_level_0__'],
        num_rows: 76
    })
    val: Dataset({
        features: ['source', 'reqs', 'class', '__index_level_0__'],
        num_rows: 76
    })
    train: Dataset({
        features: ['source', 'reqs', 'class', '__index_level_0__'],
        num_rows: 352
    })
})

The dataset contains 5 different labels:
* `F` - Functional requirement
* `O` - Operations requirement
* `PE` - Performance requirement
* `SE` - Security requirement
* `US` - Usability requirement

In [ ]:
ds['train'].features['class'].names

['F', 'O', 'PE', 'SE', 'US']

In [ ]:
num_labels = ds['train'].features['class'].num_classes

Select any of the six LMs and import the corresponding tokenizer.

* `bert-base-cased`
* `roberta-base`
* `allenai/scibert_scivocab_cased`
* `limsc/reqbert-tapt-epoch29`
* `limsc/reqroberta-tapt-epoch43`
* `limsc/reqscibert-tapt-epoch20`

In [ ]:
model_checkpoint = 'limsc/reqroberta-tapt-epoch43'

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    model_type_dict[model_checkpoint],
    use_fast = True
)

def encode(requirements):
    return tokenizer(requirements['reqs'], truncation = True, max_length = 128)

tokenized_ds = ds.map(encode, batched = True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_ds

DatasetDict({
    test: Dataset({
        features: ['source', 'reqs', 'class', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 76
    })
    val: Dataset({
        features: ['source', 'reqs', 'class', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 76
    })
    train: Dataset({
        features: ['source', 'reqs', 'class', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 352
    })
})

# Model Fine-tuning (Single Loop)

Pre-batch datasets in to batches of specified size.

In [ ]:
from transformers import DataCollatorWithPadding

batch_size = 32
output_col = 'class'

data_collator = DataCollatorWithPadding(
    tokenizer = tokenizer,
    return_tensors = 'tf'
)

def batching(tokenized_ds, batch_size):

  batched_train_ds = tokenized_ds['train'].to_tf_dataset(
      columns = ['attention_mask', 'input_ids', 'token_type_ids'],
      label_cols = [output_col],
      shuffle = False,
      drop_remainder = False,
      collate_fn = data_collator,
      batch_size = batch_size
  )

  batched_val_ds = tokenized_ds['val'].to_tf_dataset(
      columns = ['attention_mask', 'input_ids', 'token_type_ids'],
      label_cols = [output_col],
      shuffle = False,
      drop_remainder = False,
      collate_fn = data_collator,
      batch_size = batch_size
  )

  batched_test_ds = tokenized_ds['test'].to_tf_dataset(
      columns = ['attention_mask', 'input_ids', 'token_type_ids'],
      label_cols = [output_col],
      shuffle = False,
      drop_remainder = False,
      collate_fn = data_collator,
      batch_size = batch_size
  )

  return batched_train_ds, batched_val_ds, batched_test_ds

batched_train_ds, batched_val_ds, batched_test_ds = batching(tokenized_ds, batch_size)

For tf2.6 onwards, the weights of the classification head is only affected by seeds set using tf.random_set_seed. See discussion on [stackoverflow](https://stackoverflow.com/questions/32419510/how-to-get-reproducible-results-in-keras).

In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification, create_optimizer

seed = 67897
tf.random.set_seed(seed)

num_epochs = 2
initial_lr = 2e-5

def create_model(num_epochs, initial_lr):

  from_pt = True if model_checkpoint == 'allenai/scibert_scivocab_cased' else False

  model = TFAutoModelForSequenceClassification.from_pretrained(
      model_checkpoint,
      num_labels = num_labels,
      from_pt = from_pt
  )

  batches_per_epoch = len(tokenized_ds['train']) // batch_size
  total_train_steps = int(batches_per_epoch * num_epochs)

  optimizer, schedule = create_optimizer(
      init_lr = initial_lr,
      num_warmup_steps = 0,
      num_train_steps = total_train_steps,
      weight_decay_rate = 0.01
  )

  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

  model.compile(
      optimizer = optimizer,
      loss = loss,
      metrics = tf.metrics.SparseCategoricalAccuracy()
  )

  return model

model = create_model(num_epochs, initial_lr)

Downloading:   0%|          | 0.00/644 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at limsc/reqroberta-tapt-epoch43 and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


This chunk of code allows you to control which of the encoder layers have been frozen.

In [ ]:
from transformers import models

for layer in model.layers[:]:
    print(layer, layer.trainable)

print('=========================================================================')

encoder_layer_name = {
    'bert-base-cased' : models.bert.modeling_tf_bert.TFBertMainLayer,
    'roberta-base' : models.roberta.modeling_tf_roberta.TFRobertaMainLayer,
    'allenai/scibert_scivocab_cased' : models.bert.modeling_tf_bert.TFBertMainLayer
}

# Adjust this to freeze the later layer
frozen_layers = []

for layer in model.layers[:]:
  
  if isinstance(layer, encoder_layer_name[model_type_dict[model_checkpoint]]):
    
    for idx, layer in enumerate(layer.encoder.layer):
      
      if idx in frozen_layers:
        layer.trainable = False
      
      # Confirm the chosen layers are frozen
      print(layer, layer.trainable)

<transformers.models.roberta.modeling_tf_roberta.TFRobertaMainLayer object at 0x7f4dbd3bdd90> True
<transformers.models.roberta.modeling_tf_roberta.TFRobertaClassificationHead object at 0x7f4dbb7dd8d0> True
<transformers.models.roberta.modeling_tf_roberta.TFRobertaLayer object at 0x7f4d369b7190> True
<transformers.models.roberta.modeling_tf_roberta.TFRobertaLayer object at 0x7f4db8904b50> True
<transformers.models.roberta.modeling_tf_roberta.TFRobertaLayer object at 0x7f4d34169750> True
<transformers.models.roberta.modeling_tf_roberta.TFRobertaLayer object at 0x7f4d34203e50> True
<transformers.models.roberta.modeling_tf_roberta.TFRobertaLayer object at 0x7f4d349f8350> True
<transformers.models.roberta.modeling_tf_roberta.TFRobertaLayer object at 0x7f4dbc862a10> True
<transformers.models.roberta.modeling_tf_roberta.TFRobertaLayer object at 0x7f4db97168d0> True
<transformers.models.roberta.modeling_tf_roberta.TFRobertaLayer object at 0x7f4db89d2a50> True
<transformers.models.roberta.mode

In [ ]:
model.summary()

Model: "tf_roberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLayer  multiple                 124055040 
 )                                                               
                                                                 
 classifier (TFRobertaClassi  multiple                 594437    
 ficationHead)                                                   
                                                                 
Total params: 124,649,477
Trainable params: 124,649,477
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import math
import os
import numpy as np
from tensorflow.keras.callbacks import Callback, CSVLogger, ModelCheckpoint
from transformers.keras_callbacks import PushToHubCallback
from sklearn.metrics import f1_score

class gridsearch(Callback):

    def on_epoch_end(self, epoch, logs):

        logs['seed'] = seed
        logs['batch_size'] = batch_size
        logs['learning_rate'] = initial_lr

gridsearch_cb = gridsearch()

csvlogger_file = f'{model_name_dict[model_checkpoint]}-{task_name_dict[ds_name]}.csv'
csvlogger_cb = CSVLogger(csvlogger_file, append = True)

In [ ]:
callbacks = [gridsearch_cb, csvlogger_cb]

In [ ]:
# model.fit(
#     batched_train_ds,
#     validation_data = batched_val_ds,
#     epochs = num_epochs,
#     callbacks = callbacks
# )

# Hyperparameter tuning

Perform a full grid search of all `batch_sizes` and `initial_lrs`.

In [ ]:
batch_sizes = [32]
initial_lrs = [5e-5, 3e-5, 2e-5]
seeds = [21916, 25412, 56281, 61712, 30488,
         28215, 78867, 87843, 67918, 93327,
         95420, 11905, 86349, 12082, 81996]

num_epochs = 5

for batch_size in batch_sizes:

  batched_train_ds, batched_val_ds, batched_test_ds = batching(tokenized_ds, batch_size)

  for initial_lr in initial_lrs:
    
    for seed in seeds:
    
      tf.random.set_seed(seed)
      model = create_model(num_epochs, initial_lr)

      frozen_layers = []

      for layer in model.layers[:]:
        
        if isinstance(layer, encoder_layer_name[model_type_dict[model_checkpoint]]):
          
          for idx, layer in enumerate(layer.encoder.layer):
            
            if idx in frozen_layers:
              layer.trainable = False

      csvlogger_file = f'{task_name_dict[ds_name]}-{model_name_dict[model_checkpoint]}.csv'
      csvlogger_cb = CSVLogger(csvlogger_file, append = True)

      callbacks = [gridsearch_cb, csvlogger_cb]
      
      model.fit(
          batched_train_ds,
          validation_data = batched_val_ds,
          epochs = num_epochs,
          callbacks = callbacks
      )

# Model Evaluation

In [ ]:
mod_hyperparam_dict = {
    'bert-base-cased' : {'batch_size' : 16, 'initial_lr' : 5e-5, 'num_epochs' : 3},
    'roberta-base' : {'batch_size' : 16, 'initial_lr' : 5e-5, 'num_epochs' : 4},
    'allenai/scibert_scivocab_cased' : {'batch_size' : 16, 'initial_lr' : 5e-5, 'num_epochs' : 3},
    'limsc/reqbert-tapt-epoch29' : {'batch_size' : 16, 'initial_lr' : 5e-5, 'num_epochs' : 3},
    'limsc/reqroberta-tapt-epoch43' : {'batch_size' : 16, 'initial_lr' : 5e-5, 'num_epochs' : 3},
    'limsc/reqscibert-tapt-epoch20' : {'batch_size' : 16, 'initial_lr' : 5e-5, 'num_epochs' : 3}
}

In [ ]:
from sklearn.metrics import classification_report

mods = list(mod_hyperparam_dict.keys())
results_list = []
seeds = [21916, 25412, 56281, 61712, 30488,
         28215, 78867, 87843, 67918, 93327,
         95420, 11905, 86349, 12082, 81996]
target_names = ds['train'].features['class'].names

for mod in mods:

    model_checkpoint = mod

    batch_size = mod_hyperparam_dict[mod]['batch_size']
    initial_lr = mod_hyperparam_dict[mod]['initial_lr']
    num_epochs = mod_hyperparam_dict[mod]['num_epochs']

    tokenizer = AutoTokenizer.from_pretrained(
        model_type_dict[mod],
        use_fast = True
    )

    tokenized_ds = ds.map(encode, batched = True)

    data_collator = DataCollatorWithPadding(
        tokenizer = tokenizer,
        return_tensors = 'tf'
    )

    batched_train_ds, batched_val_ds, batched_test_ds = batching(tokenized_ds, batch_size)

    for seed in seeds:

        tf.random.set_seed(seed)
        model = create_model(num_epochs, initial_lr)

        model.fit(
            batched_train_ds,
            epochs = num_epochs
        )

        y_true = tokenized_ds['test'][output_col]
        y_pred = np.argmax(model.predict(batched_test_ds)['logits'], axis = 1)
        
        rep = classification_report(
            y_true,
            y_pred,
            target_names = target_names,
            output_dict = True
        )

        result_dict = {
            'model': mod,
            'seed' : seed,
            'F_p' : format(rep['F']['precision'], '.4f'),
            'F_r' : format(rep['F']['recall'], '.4f'),
            'F_f1' : format(rep['F']['f1-score'], '.4f'),
            'O_p' : format(rep['O']['precision'], '.4f'),
            'O_r' : format(rep['O']['recall'], '.4f'),
            'O_f1' : format(rep['O']['f1-score'], '.4f'),
            'PE_p' : format(rep['PE']['precision'], '.4f'),
            'PE_r' : format(rep['PE']['recall'], '.4f'),
            'PE_f1' : format(rep['PE']['f1-score'], '.4f'),
            'SE_p' : format(rep['SE']['precision'], '.4f'),
            'SE_r' : format(rep['SE']['recall'], '.4f'),
            'SE_f1' : format(rep['SE']['f1-score'], '.4f'),
            'US_p' : format(rep['US']['precision'], '.4f'),
            'US_r' : format(rep['US']['recall'], '.4f'),
            'US_f1' : format(rep['US']['f1-score'], '.4f'),
            'macro_p' : format(rep['macro avg']['precision'], '.4f'),
            'macro_r' : format(rep['macro avg']['recall'], '.4f'),
            'macro_f1' : format(rep['macro avg']['f1-score'], '.4f'),
            'weight_p' : format(rep['weighted avg']['precision'], '.4f'),
            'weight_r' : format(rep['weighted avg']['recall'], '.4f'),
            'weight_f1' : format(rep['weighted avg']['f1-score'], '.4f')
        }

        results_list.append(result_dict)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
22/22 [==============================] - 23s 157ms/step - loss: 1.3551 - sparse_categorical_accuracy: 0.4773
Epoch 2/3
22/22 [==============================] - 3s 157ms/step - loss: 1.0980 - sparse_categorical_accuracy: 0.6080
Epoch 3/3
22/22 [==============================] - 3s 157ms/step - loss: 0.8163 - sparse_categorical_accuracy: 0.7386


...

Epoch 1/3
22/22 [==============================] - 24s 155ms/step - loss: 1.2104 - sparse_categorical_accuracy: 0.5426
Epoch 2/3
22/22 [==============================] - 3s 154ms/step - loss: 0.5463 - sparse_categorical_accuracy: 0.8722
Epoch 3/3
22/22 [==============================] - 3s 154ms/step - loss: 0.2516 - sparse_categorical_accuracy: 0.9688


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at limsc/reqscibert-tapt-epoch20 and are newly initialized: ['classifier', 'bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
22/22 [==============================] - 24s 155ms/step - loss: 1.1272 - sparse_categorical_accuracy: 0.5852
Epoch 2/3
22/22 [==============================] - 3s 154ms/step - loss: 0.4417 - sparse_categorical_accuracy: 0.9034
Epoch 3/3
22/22 [==============================] - 3s 154ms/step - loss: 0.2084 - sparse_categorical_accuracy: 0.9659


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at limsc/reqscibert-tapt-epoch20 and are newly initialized: ['classifier', 'bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
22/22 [==============================] - 23s 155ms/step - loss: 1.1773 - sparse_categorical_accuracy: 0.5540
Epoch 2/3
22/22 [==============================] - 3s 156ms/step - loss: 0.4847 - sparse_categorical_accuracy: 0.8892
Epoch 3/3
22/22 [==============================] - 3s 156ms/step - loss: 0.2186 - sparse_categorical_accuracy: 0.9744


In [ ]:
import pandas as pd

test_df = pd.DataFrame(results_list)
test_filename = f'{task_name_dict[ds_name]}-test.csv'
test_df.to_csv(test_filename)